# Run the code block below before either model

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
                                     GlobalAveragePooling2D, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2


# Enable memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Load data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train.npy')

# Split the data
train_df, test_df, train_df_labels, test_df_labels = train_test_split(data_train.T, labels_train, test_size=0.2, random_state=42)
X_train = train_df.reshape(-1, 300, 300, 3)
X_test = test_df.reshape(-1, 300, 300, 3)

# Label encoding
lb = LabelBinarizer()
y_train = lb.fit_transform(train_df_labels)
y_test = lb.transform(test_df_labels)  # Use transform here, not fit_transform

# Code block for better model that crashes

In [ ]:
# 89% accuracy but crashes

# Data augmentation
data_gen = ImageDataGenerator(
    rotation_range=30,  # Allow rotation for augmenting rotated images
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # Shear might simulate different hand angles
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load pre-trained ResNet50 model as a base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Unfreeze the entire base model
base_model.trainable = True

# Add regularization with L2
regularizer = l2(1e-5)

# Add new layers on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=regularizer),
    BatchNormalization(),
    Dropout(0.5),
    Dense(9, activation='softmax', kernel_regularizer=regularizer)  # 9 classes for ASL A-I
])

# Compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)
checkpoint = ModelCheckpoint('best_model_resnet.h5', monitor='val_loss', save_best_only=True)

# Train the model with data augmentation
batch_size = 8 

history = model.fit(
    data_gen.flow(X_train, y_train, batch_size=batch_size), 
    epochs=50, 
    validation_data=(X_test, y_test), 
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Load the best model
best_model = tf.keras.models.load_model('best_model_resnet.h5')

# Evaluate on the validation set
val_loss, val_accuracy = best_model.evaluate(X_test, y_test)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

2023-11-29 14:54:57.376228: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 14:54:58.725513: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78911 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-11-29 14:54:58.727318: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78911 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


Epoch 1/50


2023-11-29 14:55:04.984113: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2023-11-29 14:55:07.150224: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


845/845 [==============================] - 95s 104ms/step - loss: 1.0619 - accuracy: 0.6866 - val_loss: 0.4903 - val_accuracy: 0.8863


# GOAT Model

In [2]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = True

regularizer = l2(1e-5)
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=regularizer),
    BatchNormalization(),
    Dropout(0.5),
    Dense(9, activation='softmax', kernel_regularizer=regularizer)  # 9 classes for ASL A-I
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)
checkpoint = ModelCheckpoint('best_model_resnet.h5', monitor='val_loss', save_best_only=True)

model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr, checkpoint])

best_model = tf.keras.models.load_model('best_model_resnet.h5')

# Evaluate on the validation set
val_loss, val_accuracy = best_model.evaluate(X_test, y_test)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

2023-11-29 15:21:59.543037: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 15:22:00.918291: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78911 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-11-29 15:22:00.920066: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78911 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


Epoch 1/50


2023-11-29 15:22:07.845676: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2023-11-29 15:22:09.788612: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


212/212 [==============================] - 28s 102ms/step - loss: 0.7003 - accuracy: 0.7994 - val_loss: 0.1948 - val_accuracy: 0.9449


/apps/tensorflow/2.7.0/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/50
212/212 [==============================] - 19s 90ms/step - loss: 0.1438 - accuracy: 0.9615 - val_loss: 0.2033 - val_accuracy: 0.9390
Epoch 3/50
212/212 [==============================] - 19s 89ms/step - loss: 0.1076 - accuracy: 0.9702 - val_loss: 0.1106 - val_accuracy: 0.9739
Epoch 4/50
212/212 [==============================] - 19s 89ms/step - loss: 0.0831 - accuracy: 0.9784 - val_loss: 0.1208 - val_accuracy: 0.9686
Epoch 5/50
212/212 [==============================] - 19s 89ms/step - loss: 0.0988 - accuracy: 0.9753 - val_loss: 0.1108 - val_accuracy: 0.9680
Epoch 6/50
212/212 [==============================] - 19s 90ms/step - loss: 0.0402 - accuracy: 0.9927 - val_loss: 0.1534 - val_accuracy: 0.9739
Epoch 7/50
212/212 [==============================] - 19s 90ms/step - loss: 0.0383 - accuracy: 0.9929 - val_loss: 0.1256 - val_accuracy: 0.9674
Epoch 8/50
212/212 [==============================] - 19s 89ms/step - loss: 0.0351 - accuracy: 0.9947 - val_loss: 0.1022 - val_accuracy: